<font size="+2">더블 Deep Q-Learning 및 Open AI Gym: 소개</font>

# Open AI 달 착륙선 환경

[오픈AI 체육관](https://gym.openai.com/)은 Python을 사용하여 RL 알고리즘을 테스트하고 벤치마킹하기 위한 표준화된 환경을 제공하는 RL 플랫폼입니다. 플랫폼 확장 및 맞춤형 환경 등록도 가능합니다.
[달착륙선](https://gym.openai.com/envs/LunarLander-v2)(LL) 환경에서는 에이전트가 위치, 방향 및 속도를 포함하는 이산 작업 공간과 저차원 상태 관찰을 기반으로 2차원에서 동작을 제어해야 합니다. 각 단계에서 환경은 새로운 상태에 대한 관찰과 긍정적이거나 부정적인 보상을 제공합니다.  각 에피소드는 최대 1,000개의 시간 단계로 구성됩니다. 다음 다이어그램은 훈련할 에이전트가 250개의 에피소드를 수행한 후 성공적인 착륙에서 선택한 프레임을 보여줍니다.

<img src="../assets/lunar_lander.png" width="700">

보다 구체적으로 에이전트는 6개의 연속 요소와 2개의 개별 요소를 포함하여 상태의 8가지 측면을 관찰합니다. 관찰된 요소를 기반으로 에이전트는 위치, 방향, 이동 속도 및 (부분적으로) 착륙했는지 여부를 파악합니다. 그러나 사용 가능한 작업을 사용하여 어디로 움직여야 하는지 모르거나 동작을 제어하는 ​​규칙을 이해한다는 의미에서 환경의 내부 상태를 관찰하지 않습니다.
각 시간 단계에서 에이전트는 네 가지 개별 작업 중 하나를 사용하여 모션을 제어합니다. 아무것도 할 수 없으며(현재 경로를 계속 유지) 주 엔진을 발사하거나(하향 동작을 줄이기 위해) 각 방향 엔진을 사용하여 왼쪽이나 오른쪽으로 조종할 수 없습니다. 연료 제한은 없습니다.
목표는 좌표 (0, 0)의 랜딩 패드에 있는 두 개의 플래그 사이에 에이전트를 착륙시키는 것이지만 패드 외부에도 착륙이 가능합니다. 에이전트는 정확한 착지 지점에 따라 패드를 향해 이동하면서 100~140 범위의 보상을 축적합니다. 그러나 대상에서 멀어지면 에이전트가 패드 쪽으로 이동하여 얻을 수 있는 보상이 무효화됩니다. 각 다리의 지면 접촉은 10점을 추가하며, 주 엔진 사용 비용은 -0.3점입니다.
에이전트가 착륙하거나 충돌하면 각각 100점을 더하거나 빼거나 1,000시간 단계 후에 에피소드가 종료됩니다. LL을 해결하려면 100회 연속 에피소드에서 평균 최소 200의 누적 보상을 달성해야 합니다.

# 딥 Q-러닝

Deep Q 학습은 심층 신경망을 사용하여 특정 상태에 대해 사용 가능한 작업의 값을 추정합니다. 이는 RL 에이전트가 픽셀 입력만으로 게임을 플레이하는 방법을 학습한 Deep Mind의 [심층 강화 학습으로 Atari 플레이하기](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)(2013)에 의해 소개되었습니다.
Deep Q-Learning 알고리즘은 $$q(s,a,\theta)\about이 되도록 상태를 동작에 매핑하는 다층 딥 Q 네트워크(DQN)의 가중치 세트를 학습하여 동작-값 함수 q를 근사화합니다. q^*(s,a)$$
알고리즘은 목표에 대한 DQN 추정치 간의 제곱 차이로 정의된 손실 함수에 경사하강법을 적용합니다.$$y_i=\mathbb{E}[r+\gamma\max_{a^\prime}Q(s^\prime, a^\prime; \theta_{i−1}\mid s,a)]$$네트워크 매개변수를 학습하기 위한 현재 상태-작업 쌍의 작업-값 추정:
$$L_i(\theta_i)=\mathbb{E}\left[\left(\overbrace{\underbrace{y_i}_{\text{Q 대상}}−\underbrace{Q(s, a; \theta)} _{\text{현재 예측}}}^{\text{TD 오류}}\right)^2 \right]$$
목표와 현재 추정치는 모두 가중치 세트에 따라 달라지며, 이는 훈련 전에 목표가 고정되는 지도 학습과의 차이점을 강조합니다.

## 확장

몇 가지 혁신을 통해 심층 Q-학습의 정확성과 수렴 속도가 향상되었습니다.- **경험 재생**은 상태, 행동, 보상 및 다음 상태 전환의 기록을 저장하고 에이전트가 ε-탐욕적인 행동을 선택하기 전에 각 시간 단계에서 네트워크 가중치를 업데이트하기 위해 이 경험에서 미니 배치를 무작위로 샘플링합니다. 이는 샘플 효율성을 높이고, 샘플의 자기 상관을 감소시키며, 로컬 최소값 또는 발산으로 이어질 수 있는 훈련 샘플을 생성하는 현재 가중치로 인해 피드백을 제한합니다.- **천천히 변화하는 대상 네트워크**는 신경망 가중치 업데이트에 대한 현재 네트워크 매개변수의 피드백 루프를 약화시킵니다. [심층 강화 학습을 통한 인간 수준의 제어](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf)(2015)에서 Deep Mind가 개발한 이 네트워크는 Q 네트워크와 동일한 아키텍처를 갖는 천천히 변화하는 대상 네트워크를 사용하지만 가중치는 주기적으로만 업데이트됩니다. 타겟 네트워크는 현재 상태 값에 대한 Q-Networks 추정을 업데이트하는 데 사용되는 다음 상태 값에 대한 예측을 생성합니다.- **이중 딥 Q-러닝**은 의도적으로 가장 높은 액션 값을 샘플링하기 때문에 액션 값을 과대평가하는 딥 Q-러닝의 편향을 해결합니다. Hado van Hasselt가 [Double Q-learning을 이용한 심층 강화 학습](https://arxiv.org/abs/1509.06461)(2015)에서 보여준 것처럼 이러한 편견은 균일하게 적용되지 않으면 학습 과정과 결과 정책에 부정적인 영향을 미칠 수 있습니다. 작업 선택에서 작업 값 추정을 분리하기 위해 DDQN(Double Deep Q-Learning)은 한 네트워크의 가중치를 사용하여 다음 상태에서 최상의 작업을 선택하고 다른 네트워크의 가중치를 사용하여 해당 작업 값을 제공합니다. 추정.

# 가져오기 및 설정

> 아래 코드에 필요한 TensorFlow를 버전 2.2로 업그레이드하는 방법에 대한 지침은 노트북 `04_q_learning_for_trading.ipynb`을 참조하세요.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
from time import time
from pathlib import Path

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# OpenAI Gym
import gym
from gym import wrappers

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set_style('whitegrid', {'axes.grid' : False})

In [4]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

Using GPU


결과를 재현할 수 있도록 무작위 시드를 설정합니다.

In [5]:
np.random.seed(42)
tf.random.set_seed(42)

## 결과 표시 도우미 기능

In [6]:
def format_time(t):
    m_, s = divmod(t, 60)
    h, m = divmod(m_, 60)
    return '{:02.0f}:{:02.0f}:{:02.0f}'.format(h, m, s)

## Docker 컨테이너에서 실행되도록 가상 디스플레이 활성화

이는 디스플레이가 없는 서버에서 실행하는 경우에만 필요합니다.

In [7]:
# from pyvirtualdisplay import Display
# virtual_display = Display(visible=0, size=(1400, 900))
# virtual_display.start()

# DDQN 에이전트 정의

[텐서플로우](https://www.tensorflow.org/)을(를) 사용하여 Double Deep Q-Network를 생성하겠습니다.

## 재생 버퍼

In [8]:
class Memory():
    def __init__(self, capacity, state_dims):
        self.capacity = capacity
        self.idx = 0

        self.state_memory = np.zeros(shape=(capacity, state_dims), 
                                     dtype=np.float32)
        self.new_state_memory = np.zeros_like(self.state_memory)

        self.action_memory = np.zeros(capacity, dtype=np.int32)
        self.reward_memory = np.zeros_like(self.action_memory)
        self.done = np.zeros_like(self.action_memory)

    def store(self, state, action, reward, next_state, done):
        self.state_memory[self.idx, :] = state
        self.new_state_memory[self.idx, :] = next_state
        self.reward_memory[self.idx] = reward
        self.action_memory[self.idx] = action
        self.done[self.idx] = 1 - int(done)
        self.idx += 1

    def sample(self, batch_size):
        batch = np.random.choice(self.idx, batch_size, replace=False)

        states = self.state_memory[batch]
        next_states = self.new_state_memory[batch]
        rewards = self.reward_memory[batch]
        actions = self.action_memory[batch]
        done = self.done[batch]
        return states, actions, rewards, next_states, done

## 에이전트 클래스

In [9]:
class DDQNAgent:
    def __init__(self,
                 state_dim,
                 num_actions,
                 gamma,
                 epsilon_start,
                 epsilon_end,
                 epsilon_decay_steps,
                 epsilon_exponential_decay,
                 learning_rate,
                 architecture,
                 l2_reg,
                 replay_capacity,
                 tau,
                 batch_size,
                 results_dir,
                 log_every=10):

        self.state_dim = state_dim
        self.num_actions = num_actions

        self.architecture = architecture
        self.l2_reg = l2_reg
        self.learning_rate = learning_rate
        self.experience = Memory(replay_capacity,
                                 state_dim)
        self.gamma = gamma
        self.tau = tau
        self.batch_size = batch_size
        self.idx = np.arange(batch_size, dtype=np.int32)

        self.online_network = self.build_model()
        self.target_network = self.build_model(trainable=False)
        self.optimizer = Adam(lr=learning_rate)
        self.update_target()

        self.epsilon = epsilon_start
        self.epsilon_decay_steps = epsilon_decay_steps
        self.epsilon_decay = (epsilon_start - epsilon_end) / epsilon_decay_steps
        self.epsilon_exponential_decay = epsilon_exponential_decay
        self.epsilon_history = []

        self.total_steps = self.train_steps = 0
        self.episodes = self.episode_length = self.train_episodes = 0
        self.steps_per_episode = []
        self.episode_reward = 0
        self.rewards_history = []

        self.results_dir = results_dir
        self.experiment = experiment
        self.log_every = log_every
        
        self.summary_writer = (tf.summary
                               .create_file_writer(results_dir.as_posix()))
        self.start = time()
        self.train = True

    def build_model(self, trainable=True):
        layers = []
        for i, units in enumerate(self.architecture, 1):
            layers.append(Dense(units=units,
                                input_dim=self.state_dim if i == 1 else None,
                                activation='relu',
                                kernel_regularizer=l2(self.l2_reg),
                                trainable=trainable))
        layers.append(Dense(units=self.num_actions,
                            trainable=trainable))
        return Sequential(layers)

    def update_target(self):
        self.target_network.set_weights(self.online_network.get_weights())

    # @tf.function
    def epsilon_greedy_policy(self, state):
        self.total_steps += 1
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.num_actions)
        q = self.online_network.predict(state)
        return np.argmax(q, axis=1).squeeze()

    # @tf.function
    def decay_epsilon(self):
        if self.train:
            if self.episodes < self.epsilon_decay_steps:
                self.epsilon -= self.epsilon_decay
            else:
                self.epsilon *= self.epsilon_exponential_decay

    def log_progress(self):
        self.rewards_history.append(self.episode_reward)
        self.steps_per_episode.append(self.episode_length)

        avg_steps_100 = np.mean(self.steps_per_episode[-100:])
        avg_steps_10 = np.mean(self.steps_per_episode[-10:])
        max_steps_10 = np.max(self.steps_per_episode[-10:])
        avg_rewards_100 = np.mean(self.rewards_history[-100:])
        avg_rewards_10 = np.mean(self.rewards_history[-10:])
        max_rewards_10 = np.max(self.rewards_history[-10:])

        with self.summary_writer.as_default():
            tf.summary.scalar('Episode Reward', self.episode_reward, step=self.episodes)
            tf.summary.scalar('Episode Rewards (MA 100)', avg_rewards_100, step=self.episodes)
            tf.summary.scalar('Episode Steps', self.episode_length, step=self.episodes)
            tf.summary.scalar('Epsilon', self.epsilon, step=self.episodes)

        if self.episodes % self.log_every == 0:
            template = '{:03} | {} | Rewards {:4.0f} {:4.0f} {:4.0f} | ' \
                       'Steps: {:4.0f} {:4.0f} {:4.0f} | Epsilon: {:.4f}'
            print(template.format(self.episodes, format_time(time() - self.start),
                                  avg_rewards_100, avg_rewards_10, max_rewards_10,
                                  avg_steps_100, avg_steps_10, max_steps_10,
                                  self.epsilon))

    def memorize_transition(self, s, a, r, s_prime, done):
        self.experience.store(s, a, r, s_prime, done)
        self.episode_reward += r
        self.episode_length += 1

        if done:
            self.epsilon_history.append(self.epsilon)
            self.decay_epsilon()
            self.episodes += 1
            self.log_progress()
            self.episode_reward = 0
            self.episode_length = 0

    def experience_replay(self):
        # not enough experience yet
        if self.batch_size > self.experience.idx:
            return

        # sample minibatch
        states, actions, rewards, next_states, done = self.experience.sample(self.batch_size)

        # select best next action (online)
        next_action = tf.argmax(self.online_network.predict(next_states, self.batch_size), axis=1, name='next_action')
        # predict next q values (target)
        next_q_values = self.target_network.predict(next_states, self.batch_size)
        # get q values for best next action
        target_q = (tf.math.reduce_sum(next_q_values *
                                       tf.one_hot(next_action,
                                                  self.num_actions),
                                       axis=1, name='target_q'))
        # compute td target
        td_target = rewards + done * self.gamma * target_q

        with tf.GradientTape() as tape:
            q_values = self.online_network(states)
            q_values = tf.math.reduce_sum(q_values * tf.one_hot(actions, self.num_actions), axis=1, name='q_values')
            loss = tf.math.reduce_mean(tf.square(td_target - q_values))

        # run back propagation
        variables = self.online_network.trainable_variables
        gradients = tape.gradient(loss, variables)

        self.optimizer.apply_gradients(zip(gradients, variables))
        with self.summary_writer.as_default():
            tf.summary.scalar('Loss', loss, step=self.train_steps)
        self.train_steps += 1

        if self.total_steps % self.tau == 0:
            self.update_target()

    def store_results(self):
        result = pd.DataFrame({'Rewards': self.rewards_history,
                               'Steps'  : self.steps_per_episode,
                               'Epsilon': self.epsilon_history},
                              index=list(range(1, len(self.rewards_history) + 1)))

        result.to_csv(self.results_dir / 'results.csv', index=False)

# 실험 실행

In [10]:
experiment = 0

In [11]:
results_dir = Path('results', 'lunar_lander', 'experiment_{}'.format(experiment))
if not results_dir.exists():
    results_dir.mkdir(parents=True)

## OpenAI Gym 달 착륙선 환경 설정

LL 환경에서 주요 매개변수를 인스턴스화하고 추출하는 것부터 시작하겠습니다.

In [12]:
env = gym.make('LunarLander-v2')
state_dim = env.observation_space.shape[0]  # number of dimensions in state
num_actions = env.action_space.n  # number of actions
max_episode_steps = env.spec.max_episode_steps  # max number of steps per episode
env.seed(42)

[42]

또한 에이전트의 성과를 표시하는 비디오를 주기적으로 저장할 수 있는 내장 래퍼를 사용합니다.

In [13]:
monitor_path = results_dir / 'monitor'
video_freq = 500

In [14]:
env = wrappers.Monitor(env,
                       directory=monitor_path.as_posix(),
                       video_callable=lambda count: count % video_freq == 0,
                      force=True)

## 하이퍼파라미터 정의

에이전트의 성능은 여러 하이퍼파라미터에 매우 민감합니다. Q-Network, 재생 버퍼 및 ε-탐욕 정책 매개변수를 설정하기 전에 할인부터 시작하겠습니다.

### 할인율

In [15]:
gamma = .99

### Q-네트워크 매개변수

In [16]:
learning_rate = 0.0001

In [17]:
architecture = (256, 256)  # units per layer
l2_reg = 1e-6  # L2 regularization

100개의 시간 단계마다 대상 네트워크를 업데이트하고, 재생 메모리에 최대 1m의 과거 에피소드를 저장하고, 에이전트를 훈련하기 위해 메모리에서 1,024개의 미니 배치를 샘플링합니다.

### 재생 버퍼 매개변수

In [18]:
tau = 100  # target network update frequency
replay_capacity = int(1e6)
batch_size = 1024

### ε-욕심 정책

ε-탐욕 정책은 ε=1에서 순수 탐색으로 시작하여 100개 에피소드에 걸쳐 모든 단계에서 0.01까지 선형적으로 감소한 다음 0.99 비율로 지수적으로 감소합니다.

In [19]:
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay_steps = 100
epsilon_exponential_decay = .99

## DDQN 에이전트 인스턴스화

In [20]:
agent = DDQNAgent(state_dim=state_dim,
                  num_actions=num_actions,
                  learning_rate=learning_rate,
                  gamma=gamma,
                  epsilon_start=epsilon_start,
                  epsilon_end=epsilon_end,
                  epsilon_decay_steps=epsilon_decay_steps,
                  epsilon_exponential_decay=epsilon_exponential_decay,
                  replay_capacity=replay_capacity,
                  architecture=architecture,
                  l2_reg=l2_reg,
                  tau=tau,
                  batch_size=batch_size,
                  results_dir=results_dir)

## 훈련 및 테스트 에이전트

In [21]:
tf.keras.backend.clear_session()

In [22]:
max_episodes = 2500
test_episodes = 0

에피소드 번호, 경과 시간 외에도 지난 100회 및 최근 10회 보상 및 에피소드 길이에 대한 이동 평균과 지난 10회 반복에 대한 각각의 최대값을 기록합니다. 우리는 또한 엡실론의 붕괴를 추적합니다.

In [23]:
while agent.episodes < max_episodes:
    this_state = env.reset()
    done = False
    while not done:
        action = agent.epsilon_greedy_policy(this_state.reshape(-1, state_dim))
        next_state, reward, done, _ = env.step(action)
        agent.memorize_transition(this_state, action, reward, next_state, done)
        agent.experience_replay()
        this_state = next_state
    if np.mean(agent.rewards_history[-100:]) > 200:
        break

agent.store_results()
env.close()

010 | 00:00:03 | Rewards -193 -193  -70 | Steps:   93   93  143 | Epsilon: 0.9010
020 | 00:00:29 | Rewards -157 -122  -59 | Steps:   95   97  153 | Epsilon: 0.8020
030 | 00:00:59 | Rewards -140 -105  -43 | Steps:   95   96  129 | Epsilon: 0.7030
040 | 00:01:37 | Rewards -129  -98  -29 | Steps:  101  116  188 | Epsilon: 0.6040
050 | 00:02:25 | Rewards -114  -51   25 | Steps:  108  137  277 | Epsilon: 0.5050


KeyboardInterrupt: 

## 결과 평가

In [ ]:
results = pd.read_csv(results_dir / 'results.csv')
results['MA100'] = results.rolling(window=100, min_periods=25).Rewards.mean()

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(16, 4), sharex=True)
results[['Rewards', 'MA100']].plot(ax=axes[0])
axes[0].set_ylabel('Rewards')
axes[0].set_xlabel('Episodes')
axes[0].axhline(200, c='k', ls='--', lw=1)
results[['Steps', 'Epsilon']].plot(secondary_y='Epsilon', ax=axes[1]);
axes[1].set_xlabel('Episodes')
fig.suptitle('Double Deep Q-Network Agent | Lunar Lander', fontsize=16)
fig.tight_layout()
fig.subplots_adjust(top=.9)
fig.savefig(results_dir / 'trading_agent_2ed', dpi=300)

## 텐서보드

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir results/lunar_lander/experiment_0